<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-case-studies/blob/master/neural_machine_translation_for_hindi_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Machine Translation for Hindi-English: Sequence to sequence learning

This article throws light on various aspects of building a basic Neural Machine Translation (NMT) model using the sequence to sequence learning approach, with LSTMs (Long Short-Term Memory). While there are numerous papers and blogs on NMT, this is another attempt on highlighting some of the intuitive features and also a step by step guide to performing similar NLP tasks. I have tried maintaining a balance between technical and non-technical details in this article. I hope it helps. So let’s get started!

Reference:

https://medium.com/analytics-vidhya/neural-machine-translation-for-hindi-english-sequence-to-sequence-learning-1298655e334a

https://github.com/richaranjan23/My_Projects/tree/master/MSc_dissertation

http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/

## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

import string
import codecs
import re
import h5py
from string import digits

from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model, get_file

import matplotlib.pyplot as plt
%matplotlib inline

## The Corpus — Pre-processing

The languages chosen for building this translation models were Hindi and English, and the parallel corpora was obtained from the [IIT Bombay Hindi-English Parallel Corpus](http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/). This is a Hindi-English parallel corpus containing 1,492,827 pairs of sentences. 

### Download the dataset

In [4]:
# ref: https://stackoverflow.com/questions/56976078/how-do-i-load-images-dataset-using-tf-keras-utils-get-file
dataset = tf.keras.utils.get_file(
    fname="parallel.tgz", 
    origin="http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/parallel.tgz", 
    extract=True,
)

96100352/96098008 [==============================] - 42s 0us/step


In [5]:
# check the donwloaded data that is donloaded into ~/.keras/datasets directory
! ls ~/.keras/datasets

parallel  parallel.tgz


In [0]:
# copy the donwloaded data into current directory
! cp -r ~/.keras/datasets .

### Original data load

In [0]:
eng_sentence = (open('datasets/parallel/IITB.en-hi.en', encoding='utf-8', errors='ignore').read()).split('\n')[:-1]
hin_sentence = (open('datasets/parallel/IITB.en-hi.hi', encoding='utf-8', errors='ignore').read()).split('\n')[:-1]

In [26]:
# prepare dataframe for both sentence
lines = pd.DataFrame(columns=['eng', 'hindi'])
lines.eng = eng_sentence
lines.hindi = hin_sentence
lines.head()

,eng,hindi
0,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,A list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...


In [27]:
lines.shape

(1561840, 2)

### Original data cleaning

In [28]:
# truncate dataset
lines = lines[:1000000]
lines.shape

(1000000, 2)

In [0]:
# clean hindi text
hindi_clean = []
for line in lines.hindi:
  nstr = re.sub('\s+_\s+[A-Za-z]', '', line)
  nstr = re.sub('\(*\)*', '', nstr)
  nstr = re.sub('%.*?[A-Za-z]', '', nstr)
  nstr = re.sub('[A-Za-z]', '', nstr)
  nstr = nstr.strip()
  nstr = re.sub('\s+', ' ', nstr)
  hindi_clean.append(nstr)

In [0]:
# clean english text
eng_clean = []
for line in lines.eng:
  estr = re.sub('\s+_\s', '', line)
  estr = re.sub(r'[^\x00-\x7F]+', ' ', estr)
  estr = estr.strip()
  estr = re.sub('\s+', ' ', estr)
  eng_clean.append(estr)

In [31]:
len(hindi_clean), len(eng_clean)

(1000000, 1000000)

In [0]:
# update dataframe hindi/english column with cleaned data
lines.hindi = hindi_clean
lines.eng = eng_clean

# make lower case
lines.hindi = lines.hindi.apply(lambda x: x.lower())
lines.eng = lines.eng.apply(lambda x: x.lower())

In [0]:
# remove punctuation
exclude = set(string.punctuation)
lines.hindi = lines.hindi.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.eng = lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# remove digits from text
remove_digits = str.maketrans('', '', digits)
lines.hindi = lines.hindi.apply(lambda x: x.translate(remove_digits))
lines.eng = lines.eng.apply(lambda x: x.translate(remove_digits))

In [35]:
lines.sample(10)

,eng,hindi
338440,and of the fruits of the date palm and grapes...,और इसी तरह खुरमें और अंगूर के फल से हम तुमको श...
748795,gautama buddha,धर्मकेतु
225478,cisco ciscosecurity,सिस्कोसिस्कोसिक्यूरिटी
827630,zoological science,जंतु विज्ञान
409257,as for those who with no authority to do so di...,जो लोग बिना किसी ऐसे प्रमाण के जो उनके पास आया...
46065,direction,दिशा
347345,and we spread out the earth and cast in it fir...,और धरती को हमने फैलाया और उसमे अटल पहाड़ डाल द...
517243,it s not just that he s spending days in israe...,आखिर बराक ओबामा इजरायल और अरब के साथ उसके संघर...
467640,if you make both figures narayana and baladeva...,यदि तुम नारायण और बलदेव दोनों की आकृति बनाओ तो...
819093,attractive,मंजु


In [36]:
lines.sample(20)

,eng,hindi
186593,gangneung,गेंगनुएंग
482230,it had warned the rbi as far back as that gol...,बीआइएस ने रिज़र्व बैंक को में ही आगाह कर दिया...
170409,spacing modifier letters,जगह परिवर्धक अक्षर
571255,this was after,यह के बाद की बात है
980329,liquidity adjustment facility can be availed o...,चलनिधि समायोजन सुविधा बैंकों द्वारा उनके चल सं...
43133,the type of the values in the interval,अंतराल में मानों का प्रकार
282026,and inquire those of the city where we have be...,आप उस बस्ती से पूछ लीजिए जहाँ हम थे और उस क़ाफ...
831924,cardboard,दफ़्ती
646312,charge time accuracy profile,समय सटीकता प्रोफाइल चार्ज करें
82013,th,आठवां


## Filter short sentences